In [1]:
import numpy as np
import json
import sys
import random
import requests
import sqlalchemy
import os
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from config import driver, username, password, host, port, database 
from slack_config import webhookURL
from refresh_from_api import refresh_from_api
from flask import Flask, render_template, jsonify, request, redirect, send_from_directory

In [2]:
#################################################
# Database Setup
#################################################

connection_string = f"{driver}://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)
connection = engine.connect()

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(connection, reflect=True)

# Save reference to the tables
ContentChoice = Base.classes.contentChoice
DailyForecastTB = Base.classes.dailyForecastTB
HourlyForecastTB = Base.classes.hourlyForecastTB

In [ ]:
@app.route("/data2/<homeLocation>", methods=["GET", "POST"])
def site_data(homeLocation=None):
    """Returns the data for the website"""

In [14]:
    #     homeLocation = request.form.get("campus") or "UCLA"

    homeLocation = "UCLA"


    # Create our session (link) from Python to the DB
    session = Session(engine)

    # Query to fetch the list of location names
    resultsLocations = session.query(ContentChoice).all()

    # if homeLocation == None:
    #     homeLocation = session.query(ContentChoice.shortName).filter_by(homePage = 'Y').one()
    #     print(homeLocation)

    # Query to fetch the list of Daily forecast for 7 days
    resultsDailyForecast = session.query(DailyForecastTB).filter_by(shortName = homeLocation).all()

    # Query to fetch the list of Hourly forecast
    resultsHourlyForecast = session.query(HourlyForecastTB).filter_by(shortName = homeLocation).all()

    # # Query to fetch the list of Daily forecast for 7 days
    # resultsDailyForecast = session.query(DailyForecastTB).all()

    # # Query to fetch the list of Hourly forecast
    # resultsHourlyForecast = session.query(HourlyForecastTB).all()

    content_choice = []
    for row in resultsLocations:
        choice = {}

        location = row.locationName
        short_name = row.shortName
        city = row.city
        latitude = row.latitude
        longitude = row.longitude

        choice["locationName"] = location
        choice["shortName"] = short_name
        choice["city"] = city
        choice["latitude"] = str(latitude)
        choice["longitude"] = str(longitude)

        content_choice.append(choice)

    daily_forecasts = []
    for row in resultsDailyForecast:
        day_forecast = {}

        day_forecast["daily_responseNumber"] = int(row.responseNumber)
        day_forecast["daily_responseName"] = row.responseName
        # Since the startDateTime is in GMT, we might not need it for the index.html. So commenting this line
        # day_forecast["daily_startDateTime"] = row.startDateTime
        day_forecast["daily_startDate"] = str(row.startDate)
        day_forecast["daily_startTime"] = str(row.startTime)
        # Since the endDateTime is in GMT, we might not need it for the index.html. So commenting this line
        # day_forecast["daily_endDateTime'] = str(row.endDateTime)
        day_forecast["daily_endDate"] = str(row.endDate)
        day_forecast["daily_endTimes"] = str(row.endTime)
        day_forecast["daily_isDaytime"] = row.isDaytime
        day_forecast["daily_temperature"] = int(row.temperature)
        # we have all units as F; do we have to send it as part of JSON
        day_forecast["daily_temperatureUnit"] = row.temperatureUnit
        # temperatureTrend always seem to be None and also not used for visualization. So commenting this line
        # day_forecast["daily_temperatureTrend"] = row.temperatureTrend)
        day_forecast["daily_windSpeed"] = row.windSpeed
        day_forecast["daily_minWindSpeed"] = int(row.minWindSpeed)
        day_forecast["daily_maxWindSpeed"] = str(row.maxWindSpeed)
        day_forecast["daily_windSpeedUnit"] = row.windSpeedUnit
        day_forecast["daily_windDirection"] = row.windDirection
        day_forecast["daily_icon"] = row.icon
        day_forecast["daily_shortForecast"] = row.shortForecast
        day_forecast["daily_detailedForecast"] = row.detailedForecast
        day_forecast["daily_retrievalDateTime"] = row.retrievalDateTime

        daily_forecasts.append(day_forecast)

    hourly_forecasts = []
    for row in resultsHourlyForecast:
        hour_forecast = {}

        hour_forecast["hourly_responseNumber"] = int(row.responseNumber)
        # Since the startDateTime is in GMT, we might not need it for the index.html. So commenting this line
        # hour_forecast["hourly_startDateTime"] = row.startDateTime
        hour_forecast["hourly_startDate"] = str(row.startDate)
        hour_forecast["hourly_startTime"] = str(row.startTime)
        # Since the endDateTime is in GMT, we might not need it for the index.html. So commenting this line
        # hour_forecast["hourly_endDateTime"] = row.endDateTime
        hour_forecast["hourly_endDate"] = str(row.endDate)
        hour_forecast["hourly_endTime"] = str(row.endTime)
        # Will we need isDaytime for any of the reporting?
        hour_forecast["hourly_isDaytime"] = row.isDaytime
        hour_forecast["hourly_temperature"] = int(row.temperature)
        # we have all units as F; do we have to send it as part of JSON
        # hour_forecast["hourly_temperatureUnits"] = row.temperatureUnit
        # temperatureTrend always seem to be None and also not used for visualization. So commenting this line
        # hour_forecast["hourly_temperatureTrend"] = row.temperatureTrend
        hour_forecast["hourly_windSpeed"] = row.windSpeed
        hour_forecast["hourly_hourWindSpeed"] = int(row.hourWindSpeed)
        hour_forecast["hourly_windSpeedUnit"] = row.windSpeedUnit
        hour_forecast["hourly_windDirection"] = row.windDirection
        hour_forecast["hourly_icon"] = row.icon
        hour_forecast["hourly_shortForecast"] = row.shortForecast

        hourly_forecasts.append(hour_forecast)

    session.close()

    data = {"locations" : content_choice, "dailyForecasts" : daily_forecasts, "hourlyForecasts" : hourly_forecasts}

In [ ]:
    return jsonify(data)

## Trying to figure out why app route will not update with latest database information

In [11]:
import pandas as pd

In [12]:
DailyForecastTblDF = pd.read_sql_table('dailyForecastTB', connection)
DailyForecastTblDF

,id,responseNumber,responseName,city,shortName,latitude,longitude,startDateTime,startDate,startTime,...,temperatureTrend,windSpeed,minWindSpeed,maxWindSpeed,windSpeedUnit,windDirection,icon,shortForecast,detailedForecast,retrievalDateTime
0,1,1,This Afternoon,Los Angeles,UCLA,34.07956,-118.44494,2021-10-21 22:00:00,2021-10-21,15:00:00,...,None,5 mph,5.0,NaN,mph,SW,https://api.weather.gov/icons/land/day/sct?siz...,Mostly Sunny,"Mostly sunny, with a high near 71. Southwest w...",2021-10-21 16:44:08
1,2,2,Tonight,Los Angeles,UCLA,34.07956,-118.44494,2021-10-22 01:00:00,2021-10-21,18:00:00,...,None,0 to 5 mph,0.0,5.0,mph,ESE,https://api.weather.gov/icons/land/night/fog?s...,Patchy Fog,"Patchy fog after 11pm. Mostly cloudy, with a l...",2021-10-21 16:44:08
2,3,3,Friday,Los Angeles,UCLA,34.07956,-118.44494,2021-10-22 13:00:00,2021-10-22,06:00:00,...,None,0 to 10 mph,0.0,10.0,mph,S,https://api.weather.gov/icons/land/day/fog/bkn...,Patchy Fog then Partly Sunny,"Patchy fog before 11am. Partly sunny, with a h...",2021-10-21 16:44:08
3,4,4,Friday Night,Los Angeles,UCLA,34.07956,-118.44494,2021-10-23 01:00:00,2021-10-22,18:00:00,...,None,5 to 10 mph,5.0,10.0,mph,S,https://api.weather.gov/icons/land/night/fog?s...,Patchy Fog,"Patchy fog after 11pm. Mostly cloudy, with a l...",2021-10-21 16:44:08
4,5,5,Saturday,Los Angeles,UCLA,34.07956,-118.44494,2021-10-23 13:00:00,2021-10-23,06:00:00,...,None,5 to 10 mph,5.0,10.0,mph,S,https://api.weather.gov/icons/land/day/fog/bkn...,Patchy Fog then Partly Sunny,"Patchy fog before 11am. Partly sunny, with a h...",2021-10-21 16:44:08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,374,10,Thursday Night,Irvine,None,33.64099,-117.84437,2021-10-29 01:00:00,2021-10-28,18:00:00,...,None,0 to 5 mph,0.0,5.0,mph,SE,https://api.weather.gov/icons/land/night/few?s...,Mostly Clear,"Mostly clear, with a low around 60.",2021-10-24 21:35:50
374,375,11,Friday,Irvine,None,33.64099,-117.84437,2021-10-29 13:00:00,2021-10-29,06:00:00,...,None,5 mph,5.0,NaN,mph,S,https://api.weather.gov/icons/land/day/few?siz...,Sunny,"Sunny, with a high near 76.",2021-10-24 21:35:50
375,376,12,Friday Night,Irvine,None,33.64099,-117.84437,2021-10-30 01:00:00,2021-10-29,18:00:00,...,None,5 mph,5.0,NaN,mph,S,https://api.weather.gov/icons/land/night/fog?s...,Patchy Fog,"Patchy fog after 11pm. Partly cloudy, with a l...",2021-10-24 21:35:50
376,377,13,Saturday,Irvine,None,33.64099,-117.84437,2021-10-30 13:00:00,2021-10-30,06:00:00,...,None,5 mph,5.0,NaN,mph,S,https://api.weather.gov/icons/land/day/fog/sct...,Patchy Fog then Mostly Sunny,"Patchy fog before 11am. Mostly sunny, with a h...",2021-10-24 21:35:50
